# Week 11 Homework

##### 1.	What scale of measurement applies to timeseries data? 


##### Is timeseries data continuous or discrete? Why?

##### 2.	What does ARIMA stand for? Break down and explain each piece in the name.


##### 3.	Using the data in the temperatures/ folder. Write a function that efficiently converts every column to a different temperature scale.


##### 4.	Write a function that efficiently fills in missing values in every column, excluding trailing NANs. That is: if there's temperature data on either side of a timestamp, it should interpolate through it. But if data stopped being collected, your function should leave nans at the end.


##### 5.	Find all cities A and B where city A is warmer than city B in the summer, but also colder than city B in the winter, on average.


##### 6.	For two of the cities you found in question 5 (there are many possible answers), clean up the data via aggregation or rolling averages and plot a clear visual. Explain why you chose the approach you did. 


##### 7.	Perform SARIMA on five cities, iterating through the cities programmatically. Explain your results.


#### a.	Graph the results


##### Bonus:  
What do GARCH and ARCH Stand for? Break down and explain each piece in the names. Explain the difference between the two. Create a GARCH and ARCH model on the same 5 cities from question 6. Explain and graph the results.
